# Welcome to spectral fitting with COSIpy classic

In this notebook, we'll perform a spectral fit on the Crab nebula using the 2016 balloon flight data.

## Import packages.

We're using the COSIpy classic functions in COSIpy_dc1.py, response_dc1.py, and fit_dc1.py.

In [ ]:
from COSIpy_dc1 import *
import response_dc1 as response
from fit_dc1 import *

## Define file names.

In [ ]:
data_dir = '../../data_products' # directory containing data & response files
filename_data = 'cosi_2016_balloon_data_crab.inc1.id1.extracted.tra.gz' # 2016 balloon data
response_filename = data_dir + '/Continuum_imaging_response.npz' # detector response
background_filename = data_dir + '/Scaled_Ling_BG_1x.npz' # background response
background_mode = 'from file'

## Define inputs.

In [ ]:
l_crab,b_crab = 184.55746, -5.78436 # Galactic longitude & latitude of Crab

ul = 3 # SNR limit for upper limits on spectral fit

## Read in data and define analysis object.

Read in the data set and create the main cosipy-classic “analysis_combined" object, which provides various functionalities to study the specified file. This cell usually takes a few minutes to run.

In [ ]:
analysis_data = COSIpy(data_dir,filename_data) # create analysis object
analysis_data.read_COSI_DataSet() # read in data

# Bin the data
The data are binned into time, energy, ϕ and FISBEL. FISBEL is a unique index which specifies the χ and ψ dimensions of the CDS.

Calling "get_binned_data()" may take several minutes, depending on the size of the dataset and the number of bins. Keep an eye on memory here: if your time bins are very small, for example, this could be an expensive operation.

In [ ]:
#Define the bin sizes
Delta_T = 7200 # time bin size in seconds
energy_bin_edges = np.array([150,  220,  325,  480,  520,  765, 1120, 1650, 2350, 3450, 5000]) # energy bin edges in keV
pixel_size = 6. # pixel size in degrees

analysis_data.dataset.time_binning_tags(time_bin_size=Delta_T) # time binning
analysis_data.dataset.init_binning(energy_bin_edges=energy_bin_edges,pixel_size=pixel_size) # energy and pixel binning
analysis_data.dataset.get_binned_data() # bin data

## Examine the shape of the binned data.

The binned data are contained in "analysis_combined.dataset.binned_data". This is a 4-dimensional object representing the 5 dimensions of the Compton data space (time, energy, ϕ, FISBEL).

This prints the shape of the binned data, the total time in the dataset, the number of time bins that have counts in them, and the number of counts in each time bin. Due to this energy range being so background dominated, the number of counts in each bin is very similar.

In [ ]:
print('Number of bins in each dimension (time, energy, ϕ, FISBEL):')
print(analysis_data.dataset.binned_data.shape)
print()
print('Total time in dataset (s):')
print(analysis_data.dataset.times.total_time)
print()
print('Number of populated time bins:')
print(analysis_data.dataset.times.n_ph)
print()
print('Number of counts in each time bin: ')
print(analysis_data.dataset.times.n_ph_t)

## Plot  raw spectrum & light curve.

In [ ]:
analysis_data.dataset.plot_raw_spectrum()
plt.xscale('log')

analysis_data.dataset.plot_lightcurve()

## Define the pointing object with the COSIpy pointing class.

The pointings refer to the direction/orientation of the telescope at each point in time. This cell usually takes a few minutes to run.

In [ ]:
pointing_data = Pointing(dataset=analysis_data.dataset) # definition of pointings (balloon stability + Earth rotation)

## Visualize the paths of the Crab through the field-of-view.

This isn't necessary for the spectral fitting, but is illustrative for understanding the pointings and exposure of the point source.

In [ ]:
plt.plot(pointing_data.zpoins[:,0]+360,pointing_data.zpoins[:,1],'o', label="COSI zenith pointing")
plt.plot(l_crab,b_crab,'*g',markersize=10, label="Crab")
plt.xlabel('Longitude [deg]')
plt.ylabel('Latitude [deg]')
plt.legend()

In [ ]:
analysis_data.plot_elevation([l_crab],[b_crab],['Crab'])

# Define a tracer.
Since background of the balloon data varies with time, here we define a "tracer" for our background model. 
The tracer normalizes the background model to the data in each time bin. 

In [ ]:
# Define tracer
tracer = np.sum(analysis_data.dataset.binned_data,axis=(1,2,3))
tracer = tracer/np.mean(tracer)

# Define the BG model.

In [ ]:
# Ling BG simulation to model atmospheric background
background_data = BG(dataset=analysis_data.dataset,mode=background_mode,filename=background_filename,tracer=tracer) # read in background

# Read in the Response Matrix

This usually takes a few minutes.

In [ ]:
# continuum response
rsp = response.SkyResponse(filename=response_filename,pixel_size=pixel_size) # read in detector response

## Explore the shape of the data space.

The shape of the response spans (Galactic latitude $b$, Galactic longitude $\ell$, Compton scattering angle $\phi$, FISBEL, energy). The shape of the data and background objects span (time, energy, Compton scattering angle, FISBEL), as explained above.

In [ ]:
print('Shape of response matrix (b, l, ϕ, FISBEL, energy):')
print(rsp.rsp.response_grid_normed_efinal.shape)
print()
print('Shape of binned data (time, energy, ϕ, FISBEL):')
print(analysis_data.dataset.binned_data.shape)
print()
print('Shape of background model (time, energy, ϕ, FISBEL):')
print(np.shape(background_data.bg_model))

# Calculate the point source response for the Crab.

In [ ]:
rsp.calculate_PS_response(analysis_data.dataset,pointing_data,l_crab,b_crab,1,background=background_data,pixel_size=pixel_size,lookup=False)

## Plot light curves for the data, background & sky models.

This is plotted for the 220-325 keV energy bin. The sky model is normalized to 1.

In [ ]:
plt.plot(np.sum(analysis_data.dataset.binned_data[:,1,:,:],axis=(1,2)), label="Data") # binned data light curve
plt.plot(np.sum(background_data.bg_model_reduced[1],axis=1), label="Background model") # background model
plt.plot(np.sum(rsp.sky_response[1],axis=1)*1000, label="Sky model") # sky model
plt.xlabel('Time Bins')
plt.ylabel('Counts per Time Bin')
plt.legend()

# Extract the spectrum for the Crab.

For each energy bin individually, this determines the coefficients for the sky and background models that best match the data. It can take a few hours to run!

In [ ]:
result_crab = fit(analysis_data.dataset,pointing_data,rsp,background_data) # create fitting object
result_crab.fit(iters=2000) # perform spectral fit using emcee (uses pointing definition, background model, & point source response)

## Plot the final count spectrum of the Crab.

Below is the spectrum (in counts/keV) of the Crab nebula!

The fitted value for the 480-520 keV bin is a bit low, which is likely due to the 511 keV line in the simulation.

The extracted spectrum data is saved as a .dat file.

In [ ]:
result_crab.plot_extracted_spectrum('crab_spectrum.dat')
result_crab.plot_extracted_spectrum('crab_spectrum.dat',ul=3)